# Contrôle du JetBot avec une manette de jeu

Dans cet exemple, nous allons apprendre à controller le Jetbot avec un GamePad connecté au navigateur internet de note machine.

### Création du contrôleur de la manette de jeu

La première chose à faire est de créer un widget de type ``Controller``, qui sera utilisé pour piloter le Jetbot. Ce widget utilise un paramètre nommé ``index``, qui permet d'identifier le numéro du contrôlleur. Cela est utile dans le cas où vous utilisez plusieurs contrôleurs. Pour connaître l'index du contrôleur, il faut faire les démarches suivantes:

1. Connecter l'adaptateur PS3 sur le port USB de la machine sur laquelle tourne ce notebook
2. Visiter le site [http://html5gamepad.com](http://html5gamepad.com) à partir du navigateur internet.  
3. Appuyer sur les boutons de la manette à utiliser
4. Noter le numéro de ``l'index`` de la manette qui répond lors de l'appui sur les boutons

Ensuite, nous pourrons créer et afficher le contrôleur à l'aide de cet index.

In [ ]:
import ipywidgets.widgets as widgets

controleur = widgets.Controller(index=0)  # Utiliser ici l'index du contrôleur

display(controleur)

Même si l'index est correct, vous verrez peut-être le message ``Connect gamepad and press any button``. Cela vient du fait que la manette n'est pas encore enregistrée sur le notebook. Pour l'enregistrer, il faut appuyer sur un bouton. Ensuite, vous verrez apparaître le widget.

### Les modes de la manette de jeu

La manette de jeu fournie avec le kit Waveshare possède deux modes de fonctionnement. Le premier est le mode PC/PS3/Android et le second est le mode Xbox360.
Par défaut, la manette est configurée en mode PC/PS3/Android. Dans ce mode, il existe deux sous-modes. Il faut appuyer sur la touche HOME pour changer de sous-mode.

Dans le **sous-mode 1** du mode PC/PS3/Android :
- La façade de la manette n'affiche qu'une seule LED,
- Le joystick droit est lié aux boutons 0, 1, 2 et 3,
- Le joystick gauche est lié aux axes 0 et 1 et ne donne que des valeurs égales à 0, +1 ou -1.

Dans le **sous-mode 2** du mode PC/PS3/Android ::
- La façade de la manette affiche deux LED,
- Le joystick droit est lié aux axes 2 et 5,
- Le joystick gauche est lié aux axes 0 et 1,
- Les valeurs données par les joystick sont comprises entre -1 et +1.
- Lorsque le joystick est tout en haut, la valeur est de -1 et lorsque le joystick est tout en bas, la valeur est de +1
- Lorsque le joystick est tout à droite, la valeur est de +1 et lorsque le joystick est tout à gauche, la valeur est de -1


Pour passer au mode Xbox360, il faut rester appuyé sur le bouton HOME pendant plus de 7 secondes.

Dans le **mode Xbox 360** :
- Le joystick gauche est lié aux axes 0 et 1,
- Le joystick droit est lié aux axes 2 et 3,
- Lorsque le joystick est tout en haut, la valeur est de -1 et lorsque le joystick est tout en bas, la valeur est de +1
- Lorsque le joystick est tout à droite, la valeur est de +1 et lorsque le joystick est tout à gauche, la valeur est de -1

### Lier le contrôleur de la manette aux moteurs du robot 

La manette est maintenant connectée mais le contrôleur n'est toujours pas attaché au robot ! Le lien le plus simple que nous pouvons faire est celui dédié aux moteurs. Nous allons connecter le moteur gauche à l'axe gauche de la manette et le droit à l'axe droit de la manette avec un lien unidirectionnel, en utilisant la fonction ``dlink``. Le lien se fait de la ``source`` vers la ``cible``.

J'ai choisi de me placer en mode XBOX 360. Si vous choisissez un autre mode, il faudra modifier la valeur des axes dans le code ci-dessous. Il y a aussi une petite subtilité : les valeurs étant inversées (+1 lorsque le joystick est complètement en bas et -1 lorsqu'il est totalement en haut), il faut ajouter une fonction lambda afin d'inverser les valeurs.

In [ ]:
from jetbot import Robot
import traitlets

robot = Robot()

# Mode Xbox 360 : joystick vertical gauche : Axe 1
#                 joystick vertical droit : Axe 3

lien_gauche = traitlets.dlink((controleur.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
lien_droit = traitlets.dlink((controleur.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

Votre robot devrait maintenant répondre aux commandes !

### Création d'un widget pour afficher des images

Pour commencer, nous allons créer un widget de type ``Image`` qui nous permettra d'afficher les images du flux vidéo de la caméra embarquée dans le jetbot. Nous allons paramétrer la hauteur ``height`` et la largeur ``width`` sur 300 pixels afin qu'elles ne prennent pas trop de place en mémoire.

> Remarque : Les paramètres de largeur et hauteur ne jouent que sur le rendu dans le navigateur. La résolution native de l'image avant le transport sur le réseau n'est pas modifiée.

In [ ]:
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)

### Création d'une instance de caméra

Aucune image ne s'affiche pour le moment, et c'est normal car il n'y a aucune valeur (le fameux attribut ``value``) d'enregistrée dans la variable image. Pour remplir cet attribut, nous allons l'attacher à l'attribut ``value`` d'une instance de caméra à l'aide d'un lien unidirectionnel.

Commençons donc par créer une instance de caméra :

In [ ]:
from jetbot import Camera

camera = Camera.instance()

### Création du lien entre l'instance de Camera et le widget Image

L'instance de Camera que nous utilisons produit des images au format RGB8 (rouge, vert, bleu, 8bit), alors que la classe Image du widget ne peut afficher que des images au format *JPEG*.
Nous avons donc besoin d'ajouter une fonction de conversion du format des images lors de la création du lien. Cette fonction s'appelle ``bgr8_to_jpeg`` :

In [ ]:
from jetbot import bgr8_to_jpeg

lien_camera = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

Vous devriez maintenant voir la vidéo s'afficher sur le widget Image créé précédemment !

### Mise en place du chien de garde

Vous pouvez maintenant actionner le robot et visualiser la vidéo prise par la caméra embarquée. Mais que faire si jamais une panne du Wifi survient ? Et bien, les moteurs risquent de malheureusement continuer à tourner ... et la caméra également.
Nous allons donc mettre en oeuvre un chien de garde permettant d'annuler les liens sur les moteurs et la vidéo en cas de coupure Wifi.

In [ ]:
from jetbot import Heartbeat

# Définition de la fonction à appeler lorsque le chien de garde change de status
# Si le status passe à l'état dead, alors on dédruit les liens et on arrête le robot
def action_chien_de_garde(change):
    if change['new'] == Heartbeat.Status.dead:
        lien_camera.unlink()
        lien_gauche.unlink()
        lien_droit.unlink()
        robot.stop()

chien_de_garde = Heartbeat(period=0.5)

# Active l'observation de l'attribut "status" du chien de garde
# Et appel de la fonction "action_chien_de_garde" si il y a un changement
chien_de_garde.observe(action_chien_de_garde, names='status')

Si le robot est déconnecté du réseau, alors il va s'arrêter. Vous pouvez alors le reconnecter en créant de nouveau les liens ci-dessous :

In [ ]:
# N'appelez ce code que si les liens ont été dédruits

lien_gauche = traitlets.dlink((controleur.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
lien_droit = traitlets.dlink((controleur.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)
lien_camera = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### Sauvegarder des prises de vues photo avec les boutons de la manette

Pour terminer, nous allons faire en sorte de sauvegarder des images lors de l'appui sur un bouton de la manette. Faisons cela par exemple avec le bouton X (bouton B2 en mode Xbox 360). Les images seront sauvegardées dans le répertoire ``images/``, avec un nom de fichier qui est garanti comme étant unique en utilisant la fonction ``uuid`` en Python. On utilise l'identifiant ``uuid1``, car il encode également la date et l'adresse MAC.

In [ ]:
import uuid
import subprocess

# Création d'un répertoire pour sauvegarder les images
subprocess.call(['mkdir', '-p', 'images'])

# Création d'un widget de type image
prise_image = widgets.Image(format='jpeg', width=300, height=300)

# Définition de la fonction à appeler lorsque le bouton est pressé
def sauvegarde_image(change):
    # Sauvegarde de l'image
    if change['new']:
        repertoire_fichier = 'images/' + str(uuid.uuid1()) + '.jpg'
        
        # Sauvagrde de l'image dans le fichier (on utilise image.value qui est au format JPEG)
        with open(repertoire_fichier, 'wb') as f:
            f.write(image.value)
            
        # Affiche l'image sauvegardée
        prise_image.value = image.value


controleur.buttons[2].observe(sauvegarde_image, names='value')

display(widgets.HBox([image, prise_image]))
display(controleur)

Avant de fermer ce notebook et de fermer le kernel, il faut arrêter la caméra.

In [ ]:
camera.stop()